In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers datasets optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.1 MB/s 
     |████████████████████████████████| 362 kB 66.6 MB/s 
     |████████████████████████████████| 308 kB 54.0 MB/s 
     |████████████████████████████████| 596 kB 36.4 MB/s 
     |████████████████████████████████| 6.6 MB 48.3 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 212 kB 74.4 MB/s 
     |████████████████████████████████| 140 kB 74.1 MB/s 
     |████████████████████████████████| 1.1 MB 65.2 MB/s 
     |████████████████████████████████| 127 kB 71.9 MB/s 
     |████████████████████████████████| 209 kB 63.1 MB/s 
     |████████████████████████████████| 81 kB 11.5 MB/s 
     |████████████████████████████████| 271 kB 73.3 MB/s 
     |████████████████████████████████| 144 kB 71.8 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |███████████████████████████████

In [ ]:
# загрузка библиотек

import os
import re
import gc
import operator
import pickle
from copy import deepcopy
from statistics import mean
import warnings

import numpy as np

from xgboost import (XGBClassifier)

from sklearn.model_selection import train_test_split, StratifiedGroupKFold,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE

import tensorflow_hub as hub

import gensim
import gensim.models as g
import gensim.downloader

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
#from sentence_transformers import SentenceTransformer

from transformers import BertTokenizer, TFBertModel, BertModel, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers.utils import logging
from datasets import load_metric
import torch
from torch import nn


import optuna
from optuna.samplers import TPESampler

# Манипулирование данными
import pandas as pd # для манипулирования данными

# Визуализация
# import plotly.express as px # для визуализации данных
import matplotlib.pyplot as plt # для отображения рукописных цифр


In [ ]:
warnings.filterwarnings('ignore')
logging.set_verbosity(logging.CRITICAL)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
def return_all_files_texts(prefix_file='./CO_fold/'):
    """
    Закачка документов в буфер памяти
    """
    file_names = sorted(os.listdir(prefix_file))
    text_file_list = []
    annotation_file_list = []
    for i in file_names:
        if '.txt' == i[-4:]:
            with open(prefix_file+i) as file:
                 text_file_list.append(file.read())
            with open(prefix_file+i[:-4]+'.ann') as file:
                 annotation_file_list.append(file.read())
    return text_file_list, annotation_file_list
def parse_ann_file(text):
    """
    Парсинг файлов анотации
    """
    text = list(filter(lambda x: x!='', text.split('\n')))
    text = [i.split('\t') for i in text]
    text = [[i[1].split(), i[2]] for i in text]
    text = [{'name': i[0][0], 'start': int(i[0][1]), 'end': int(i[0][-1]), 'text': i[1]} for i in text]
    return text 
def highlight_description(ann_text, raw_text):
    """
    Выделение текста операторскими скобками
    """
    filtered_list = list(filter(lambda x: x['name']=='Description', ann_text))
    filtered_list = sorted(filtered_list, key=lambda x: x['end'], reverse=True) 
    for i in filtered_list:
        if raw_text[i['end']]=='.':
            raw_text = raw_text[:i['end']]+'{__enddesc__}'+raw_text[i['end']:]
        else:
            k = 1
            while raw_text[i['end'] - k]==' ' or raw_text[i['end'] - k]=='\t':
                k += 1
            raw_text = raw_text[:i['end']-1]+'{__enddesc__}'+raw_text[i['end']-1:]
        raw_text = raw_text[:i['start']-1]+'{__startdesc__}'+raw_text[i['start']:]
    return raw_text
def replace_all(text, dic):
    """
    Множественная замена в строке
    """
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
def soft_clean(text):
    """
    очистка текста от операторов, которые не влияют на предложения
    """
    reg = r'(\\)(\w+)(=\w+|\W*|(\[\w+\])+(\{\w+\})+|(\{\w+\}+)+)\s*\n*|((\\)(\w+))+\s*\n|(\%+\s*\n)'
    dict_for_repl = {'a.k.a.': 'aka', 'e.g.':'eg', 'resp.':'resp', 't.s.':'ts', 't.i.': 'ti'}
    cleaned = text.split('\n')
    cleaned = list(filter(lambda x: not re.fullmatch(reg, x), cleaned))
    return replace_all(' '.join(cleaned).lower(), dict_for_repl)
def start_of_text(raw_text, start_of_raw):
    """
    возвращает текст с начала содержания статьи
    """
    return raw_text[raw_text.find(start_of_raw):]
def add_clean(text):
    """
    дополнительная очистка и подготовка датасета
    """
    dict_for_repl = {'}_{':'_', '\\{':' ', '\\}':' ', '{':' ', '}':' ', '$': ' $ ', ' \\ ':' ', '\[':' \[ ', '\]':' \] '}
    return re.sub(' +', ' ', replace_all(text, dict_for_repl))
def hamming_distance(list1,list2):
    """
    Просмотр ошибок через расстояние Хэмминга
    """
    result =0
    for x,(i,j) in enumerate(zip(list(list1),list2)):
        if i!=j:
            print(f'char not math{i,j}in {x}')
            result+=1
    print(f"Расстояние Хэмминга = {result}")
def vectorize_sentence(sentence,model):
    """
    Модель для работы с векторными представляниями GloVe и Word2Vec
    """
    nlp = English()
    tokenizer = Tokenizer(nlp.vocab)
    a = []
    for i in tokenizer(sentence):
        try:
            a.append(model.get_vector(str(i)))
        except:
            pass
    a=np.array(a).mean(axis=0)
    a = np.zeros(300) if np.all(a!=a) else a
    return a


In [ ]:
# вывод списка статей
print(sorted(os.listdir('/content/drive/MyDrive/Диплом/CO_fold')))

['.raw0.ann.swo', '.stats_cache', 'annotation.conf', 'kb_shortcuts.conf', 'raw.ann.bak', 'raw0.ann', 'raw0.txt', 'raw1.ann', 'raw1.txt', 'raw11.ann', 'raw11.txt', 'raw12.ann', 'raw12.txt', 'raw14.ann', 'raw14.txt', 'raw2.ann', 'raw2.txt', 'raw3.ann', 'raw4.ann', 'raw4.txt', 'raw5.ann', 'raw5.txt', 'raw6.ann', 'raw6.txt', 'raw7.ann', 'raw7.txt', 'raw8.ann', 'raw8.txt', 'raw9.ann', 'raw9.txt', 'visual.conf']


In [ ]:
# Загрузка файлов

# считывание файлов датасета
text_file_list, annotation_file_list = return_all_files_texts('/content/drive/MyDrive/Диплом/CO_fold/')

# символы начала статей
start_of_raw_list = ["{__startdesc__}Given a positive integer $n \in \mathbb{Z}_{+}$",
"The purpose of this paper is to give a class of reconstructible graphs.",
"{__startDesc__}Let $B$ be properly $n$- colored bipartite multigraph with $n+1$ edges of each color{__endDesc__}. ",
"Exercise VII.47 of [PS] (brought to my attention by Richard Stanley),",
"Let $R=",
"Let $G$ be a simple graph.{__startDesc__}The collection $D(G)=(G_v)_{v\in V(G)}$",
"{__startDesc__}A",
"{__startDesc__}Stern's diatomic sequence $a_1=1,  a_{2n}=a_n,  a_{2n+1}=a_n+a_{n+1}$",
"We prove a classification theorem for Hankel weighing matrices.",
"This billet should be regarded as a footnote to \cite{GL}",
"Since  very few papers  concern  maximal symplectic partial spreads in dimension $>4$",
"{__startDesc__}The Rogers-Ramanujan identities"]
start_of_raw_list = [i.lower() for i in start_of_raw_list]

# обработка предложений
for i in range(len(text_file_list)):
    raw = text_file_list[i]
    ann = parse_ann_file(annotation_file_list[i])
    raw = highlight_description(ann, raw)
    raw = soft_clean(raw)
    text_file_list[i] = start_of_text(raw, start_of_raw_list[i])
    text_file_list[i] = text_file_list[i].split('.')

# создание списков меток класса
all_sent = []
labels = []
for i in text_file_list:
    for j in i:
        if j.find('{__startdesc__}')!=-1  and j.find('{__enddesc__}')!=-1:
            all_sent.append(j.replace('{__startdesc__}', '').replace('{__enddesc__}', ''))
            labels.append(1)
        else:
            all_sent.append(j)
            labels.append(0)

# дополнительно очищаем датасет
all_sent = [add_clean(i) for i in all_sent]
# закрываем операторские скобки
for i in range(len(all_sent)):
    if all_sent[i].count('$')%2 == 1:
        all_sent[i] += '$'
        all_sent[i+1] = '$' + all_sent[i+1]
    

# вывод количества примеров, количество положительных меток, пример меток
print(labels.__len__())
print(sum(labels))
print(labels[:15])

del text_file_list, annotation_file_list
gc.collect()

# MathBert



# 0.9765287214329833

# X_train, X_test, y_train, y_test = train_test_split(all_sent, labels, test_size=0.33, random_state=42)

3991
77
[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]


95

In [ ]:
all_sent_np = np.array(all_sent)
labels_np = np.array(labels)

In [ ]:
class my_own_data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)



In [ ]:
metric = load_metric('glue', 'mrpc')
# metric1 = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tbs17/MathBERT')
all_encodings = tokenizer(all_sent, padding=True, truncation=True,max_length=512, add_special_tokens = True)
all_dataset = my_own_data(all_encodings, labels)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/569 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.019293410172889, 0.980706589827111], device='cuda:0'))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
def objective(trial: optuna.Trial):
    global all_sent, all_sent_np, labels_np, labels, tokenizer, all_dataset
    training_args = TrainingArguments(         
        output_dir='test_trainer',                 
        learning_rate= 0.0028290499881645993, #trial.suggest_loguniform('learning_rate', low=4e-5, high=0.01),
        weight_decay= 4.4856142312612174e-05, #trial.suggest_loguniform('weight_decay', 4e-5, 0.01),
        num_train_epochs=15,#trial.suggest_int('num_train_epochs', low = 5, high= 20),         
        per_device_train_batch_size=64,         
        per_device_eval_batch_size=8,         
        disable_tqdm=True,
        logging_dir='./logs',  
        # evaluation_strategy="epoch",
        report_to="none"
    ) 
    skf = StratifiedKFold(5, shuffle=True, random_state=33)
    for train_index, test_index in  skf.split(all_sent_np, labels_np):
        model = AutoModelForSequenceClassification.from_pretrained("tbs17/MathBERT", num_labels=2)
        X_train, X_test, y_train, y_test = all_sent_np[train_index], all_sent_np[test_index], labels_np[train_index], labels_np[test_index]
        X_train, X_test, y_train, y_test  = X_train.tolist(), X_test.tolist(), y_train.tolist(), y_test.tolist()        
        train_encodings = tokenizer(X_train, padding=True, truncation=True,max_length=512, add_special_tokens = True)
        test_encodings = tokenizer(X_test, padding=True, truncation=True,max_length=512, add_special_tokens = True)
        train_dataset = my_own_data(train_encodings, y_train)
        test_dataset = my_own_data(test_encodings, y_test)
        for param in model.bert.parameters():
            param.requires_grad = False
        model.to('cuda')
        acc_list = []
        f1_list = []
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            compute_metrics=compute_metrics,
        )
        trainer.train()
        f1 = trainer.evaluate(all_dataset)['eval_f1']
        f1_list.append(f1)
        acc = trainer.evaluate(all_dataset)['eval_accuracy']
        acc_list.append(acc)
        # gc.collect()
        print('_________')
    return mean(f1_list), mean(acc_list)

In [ ]:
def objective(trial: optuna.Trial):
    global all_sent, all_sent_np, labels_np, labels, tokenizer, all_dataset
    training_args = TrainingArguments(         
        output_dir='test_trainer',                 
        learning_rate= 0.0028290499881645993, #trial.suggest_loguniform('learning_rate', low=4e-5, high=0.01),
        weight_decay= 4.4856142312612174e-05, #trial.suggest_loguniform('weight_decay', 4e-5, 0.01),
        num_train_epochs=15,#trial.suggest_int('num_train_epochs', low = 5, high= 20),         
        per_device_train_batch_size=64,         
        per_device_eval_batch_size=8,         
        disable_tqdm=True,
        logging_dir='./logs',  
        # evaluation_strategy="epoch",
        report_to="none"
    ) 
    skf = StratifiedKFold(5, shuffle=True, random_state=33)
    for train_index, test_index in  skf.split(all_sent_np, labels_np):
        model = AutoModelForSequenceClassification.from_pretrained("tbs17/MathBERT", num_labels=2)
        X_train, X_test, y_train, y_test = all_sent_np[train_index], all_sent_np[test_index], labels_np[train_index], labels_np[test_index]
        X_train, X_test, y_train, y_test  = X_train.tolist(), X_test.tolist(), y_train.tolist(), y_test.tolist()        
        train_encodings = tokenizer(X_train, padding=True, truncation=True,max_length=512, add_special_tokens = True)
        test_encodings = tokenizer(X_test, padding=True, truncation=True,max_length=512, add_special_tokens = True)
        train_dataset = my_own_data(train_encodings, y_train)
        test_dataset = my_own_data(test_encodings, y_test)
        for param in model.bert.parameters():
            param.requires_grad = False
        model.to('cuda')
        accuracy_list = []
        trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            # eval_dataset=test_dataset,
            compute_metrics=compute_metrics,
        )
        trainer.train()
        acc = trainer.evaluate(test_dataset)['eval_f1']
        accuracy_list.append(acc)
        # gc.collect()
        print('_________')
    return mean(accuracy_list)  

In [ ]:
sampler = TPESampler(seed=10) 
study = optuna.create_study( direction="maximize",sampler=sampler)
study.optimize(objective, n_trials=1)
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
fig  = optuna.visualization.plot_param_importances(
study, target=lambda t: t.values[0], target_name="flops"
)
fig.show()

[I 2022-06-19 20:39:12,650] A new study created in memory with name: no-name-4fada734-e397-4641-a593-b42774982c9d


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

***** Running training *****
  Num examples = 3192
  Num Epochs = 15
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 750
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json


{'loss': 0.4621, 'learning_rate': 0.0009430166627215331, 'epoch': 10.0}


Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 799
  Batch size = 8


...

Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 798
  Batch size = 8


{'train_runtime': 1811.6233, 'train_samples_per_second': 26.438, 'train_steps_per_second': 0.414, 'train_loss': 0.4624228312174479, 'epoch': 15.0}


[I 2022-06-19 23:12:49,015] Trial 0 finished with value: 0.3174603174603175 and parameters: {}. Best is trial 0 with value: 0.3174603174603175.


{'eval_loss': 0.20176765322685242, 'eval_accuracy': 0.9461152882205514, 'eval_f1': 0.3174603174603175, 'eval_runtime': 20.1252, 'eval_samples_per_second': 39.652, 'eval_steps_per_second': 4.969, 'epoch': 15.0}
_________
Number of finished trials: 1
Best trial:
  Value: 0.3174603174603175
  Params: 


ValueError: ignored